<a href="https://colab.research.google.com/github/UW-ERSL/MaTruss/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch
import os, sys
import time
import matplotlib.pyplot as plt
import matplotlib
from scipy.spatial import ConvexHull
from matplotlib.patches import Polygon
sys.path.append(os.path.realpath('./src/'))
from utilFuncs import to_np, to_torch
from materialEncoder import MaterialEncoder


import seaborn as sns
import scipy as sp

### Read material properties from the database. The properties are then scaled to facilitate training

In [2]:
def preprocessData():
  #Chooses the database to extract the properties from
  df = pd.read_excel('./data/MaterialDatabase_FULL.xlsx')
  #Gets name, class and class number of each material
  dataIdentifier = {'name': df[df.columns[0]], 'className':df[df.columns[1]], 'classID':df[df.columns[2]]}
  #Gets in logarithmic from the values of the properties of each material
  trainInfo = np.log10(df[df.columns[3:]].to_numpy())
  #Scales the properties and normalize them to train the VAE equally among all properties
  dataScaleMax = torch.tensor(np.max(trainInfo, axis = 0))
  dataScaleMin = torch.tensor(np.min(trainInfo, axis = 0))
  normalizedData = (torch.tensor(trainInfo) - dataScaleMin)/(dataScaleMax - dataScaleMin)
  #Gets the normalized values
  trainingData = torch.tensor(normalizedData).float()
  #Gets an array with the minimum and maximum values of each properties
  dataInfo = {'youngsModulus':{'idx':0,'scaleMin':dataScaleMin[0], 'scaleMax':dataScaleMax[0]},\
              'costPerKg':{'idx':1,'scaleMin':dataScaleMin[1], 'scaleMax':dataScaleMax[1]},\
              'massDensity':{'idx':2,'scaleMin':dataScaleMin[2], 'scaleMax':dataScaleMax[2]},\
              'yieldStrength':{'idx':3,'scaleMin':dataScaleMin[3], 'scaleMax':dataScaleMax[3]},
              'co2Kg':{'idx':4,'scaleMin':dataScaleMin[4], 'scaleMax':dataScaleMax[4]},
              'energyKg':{'idx':5,'scaleMin':dataScaleMin[5], 'scaleMax':dataScaleMax[5]},
              'waterKg':{'idx':6,'scaleMin':dataScaleMin[6], 'scaleMax':dataScaleMax[6]}}
    
  return trainingData, dataInfo, dataIdentifier, trainInfo, df
trainingData, dataInfo, dataIdentifier, trainInfo, df = preprocessData()
#Gets number of materials and number of properties
numMaterialsInTrainingData, numFeatures = trainingData.shape
props = df.columns[3:].tolist()

C:\Users\luisy\AppData\Local\Temp\ipykernel_10396\2671264124.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainingData = torch.tensor(normalizedData).float()


### VAE training

In [ ]:
#Defines the dimension of the latent space and the number of neurons
latentDim, hiddenDim = 2, 250
#Defines a maximum of itrations
numEpochs = 50000
#Defines a klFactor
klFactor = 5e-5
#Defines a learning rate
learningRate = 2e-3
#Defines the stopping criteria
epsilon=4.1e-2
#Sets where to save the vaeNet
savedNet = './data/vaeNet.nt'
vaeSettings = {'encoder':{'inputDim':numFeatures, 'hiddenDim':hiddenDim,\
                                          'latentDim':latentDim},\
               'decoder':{'latentDim':latentDim, 'hiddenDim':hiddenDim,\
                                          'outputDim':numFeatures}}
#Loads the encoder from src\materialEncoder.py
materialEncoder = MaterialEncoder(trainingData, dataInfo, dataIdentifier, vaeSettings)

start = time.perf_counter()
#Runs the the trainAutoencoder function from src\materialEncoder.py
convgHistory = materialEncoder.trainAutoencoder(numEpochs, klFactor, savedNet, learningRate,epsilon)
print('training time : {:.2F}'.format(time.perf_counter() - start))

Iter 0 reconLoss 4.68E+01 klLoss 5.67E-03 loss 4.68E+01
Iter 500 reconLoss 2.33E-01 klLoss 6.02E-02 loss 2.93E-01
Iter 1000 reconLoss 1.08E-01 klLoss 5.91E-02 loss 1.67E-01
Iter 1500 reconLoss 7.33E-02 klLoss 5.82E-02 loss 1.32E-01
Iter 2000 reconLoss 6.02E-02 klLoss 5.65E-02 loss 1.17E-01
Iter 2500 reconLoss 4.35E-02 klLoss 5.46E-02 loss 9.81E-02
Iter 3000 reconLoss 4.59E-02 klLoss 5.28E-02 loss 9.87E-02
Iter 3500 reconLoss 3.41E-02 klLoss 5.15E-02 loss 8.56E-02
Iter 4000 reconLoss 3.73E-02 klLoss 5.02E-02 loss 8.75E-02
Iter 4500 reconLoss 3.03E-02 klLoss 4.89E-02 loss 7.91E-02
Iter 5000 reconLoss 2.83E-02 klLoss 4.79E-02 loss 7.62E-02
Iter 5500 reconLoss 2.79E-02 klLoss 4.69E-02 loss 7.48E-02
Iter 6000 reconLoss 3.14E-02 klLoss 4.57E-02 loss 7.71E-02
Iter 6500 reconLoss 2.66E-02 klLoss 4.51E-02 loss 7.17E-02


In [ ]:
plt.close('all')

## PLots loss evolution

In [ ]:
# Moving average convergence plots
def plotConvergence(convg):
  plt.figure();
  strokes = ['--', '-.', '-', ':']
  for ctr, key in enumerate(convg):
    
    y = np.array(convg[key])
    y_mvavg = np.convolve(y, np.ones(1), 'valid') / 1.
    plt.semilogy(y_mvavg, strokes[ctr], label = str(key))
    plt.xlabel('Iterations')
    plt.ylabel(str(key))
    plt.grid('True')
    plt.legend()
    plt.savefig('./figures2D/convergence.pdf')

plotConvergence(convgHistory)

## Measure differences between input and output of the trained VAE

In [ ]:
matidxs = np.array([2,11,19,45,53,68,73,4,77]).astype(int)-2
#props = ['youngsModulus','costPerKg','massDensity','yieldStrength']
print([dataIdentifier['name'][i] for i in matidxs])
print('\t \t ------TRUE DATA----------')
print('material name', end = '\t')
for p in props:
    print(p, end = '\t')
for i in matidxs:
  print(f"\n {dataIdentifier['name'][i]} \t ", end = '')
  for p in props:
    idx = materialEncoder.dataInfo[p]['idx']
    print('\t {:.2E}'.format(10.**trainInfo[i,idx]),end='')

def unnormalize(val, minval ,maxval):
  return 10.**(minval + (maxval-minval)*val)
def decodeAll():
  vae = materialEncoder.vaeNet
  decoded = vae.decoder(vae.encoder.z)
  matProp = {'youngsModulus':None,'costPerKg':None,'massDensity':None,'yieldStrength':None}
  for k in props:
    idx = materialEncoder.dataInfo[k]['idx']
    scaleMax = materialEncoder.dataInfo[k]['scaleMax']
    scaleMin = materialEncoder.dataInfo[k]['scaleMin']
    matProp[k] = unnormalize(decoded[:,idx], scaleMin ,scaleMax)
  return matProp

matProp = decodeAll()
print('\n \n \t \t ------RECONSTRUCTED DATA----------') 
print('material name', end = '\t')
for p in props:
    print(p, end = '\t')
  
for i in matidxs:
  print(f"\n {dataIdentifier['name'][i]} \t ", end = '')
  for p in props:
    print('\t {:.2E}'.format(matProp[p][i]), end='')


merr = -1000000000.
maxError = {key: merr for key in props}


print('\n \n \t \t ------RECON ERROR (%)----------') 
print('material name', end = '\t')
acum_error=np.zeros(len(props))

for p in props:
    print(p, end = '\t')
for i in range(trainInfo.shape[0]):
  if(i in matidxs): #
    print(f"\n {dataIdentifier['name'][i]} \t ", end = '')
  for p in props:
    idx = materialEncoder.dataInfo[p]['idx']
    trueData = 10**trainInfo[i,idx]
    reconData = matProp[p][i]
    err = torch.abs(100.*(trueData - reconData)/trueData)
    acum_error[idx]=acum_error[idx]+err
    if(err > maxError[p]):
      maxError[p] = err
    if(i in matidxs):
      print('\t {:.1F}'.format(err), end='')
      
print('\n\033[1;31m max Error \t \033[0m', end = '')
for p in props:
  print('\033[1;31m \t {:.1F} \033[0m'.format(maxError[p]), end='')
print('\n\033[1;31m avg Error \t \033[0m', end='')
for p in props:
  idx = materialEncoder.dataInfo[p]['idx']
  print('\033[1;31m \t {:.1F} \033[0m'.format(acum_error[idx]/trainInfo.shape[0]), end='')

## Measures distance between materials based on their euclidean distance in the latent space

In [ ]:
# distance matrix between a select candidate of materials
def drawDistanceMatrix():
  #List of materials, which are by default 3 steels, 3 aluminiums and 3 plastics
  matidxs = np.array([2,11,19,45,53,68,73,4,77]).astype(int)-2 # the rows of mats to consider in the db
  distMatrix = np.zeros((len(matidxs),)*2)
  sns.set_style("dark")
  #Loads the VAE
  vae = materialEncoder.vaeNet
  #Loads latent space coordinates of the materials
  z_np = to_np(vae.encoder.z)
  for rw in range(len(matidxs)):
    for col in range(rw, len(matidxs)):
      mx, my = matidxs[rw], matidxs[col]
      #Gets the euclidean norm from the z coordinates
      distMatrix[rw,col] = np.linalg.norm(z_np[mx,:] - z_np[my,:])
      distMatrix[col,rw] = distMatrix[rw,col]
  #Gets the maximum value of distance so that is used to scale the maximum value to 1
  maxval = np.max(distMatrix)

  distMatrix = distMatrix/maxval

  matplotlib.rcParams['figure.figsize'] = (15, 15)
  plt.matshow(distMatrix,cmap="coolwarm", alpha = 0.75)

  ax = plt.gca()

  # Set the plot labels
  xlabels = [dataIdentifier['name'][i] for i in matidxs]
  ax.set_xticks(range(len(matidxs)))
  ax.set_xticklabels(xlabels, rotation = 90, size = 12)
  ax.set_yticks(range(len(matidxs)))
  ax.set_yticklabels(xlabels, size = 12)
  plt.grid(which='minor',c='indigo', ls='-', lw='5.8')
  

  #Add text to the plot showing the values at that point
  for i in range(len(matidxs)):
      for j in range(i,len(matidxs)):
          pltText = '{:.2F}'.format(distMatrix[i,j])
          plt.text(j,i, pltText, horizontalalignment='center', \
                     verticalalignment='center', size = 12)
  plt.savefig('./figures2D/distanceMatrix.pdf',bbox_inches='tight', dpi = 200)
  plt.show()
  

  #Now gets an average distance among the 3 materials of each group
  avgMatrix = np.zeros((3,3))
  for i in range(3):
    for j in range(3):
      avgMatrix[i,j] = np.mean(distMatrix[3*i:3*(i+1), 3*j:3*(j+1)])
  
  
  
  plt.matshow(avgMatrix,cmap="coolwarm", alpha = 0.75)
  ax = plt.gca()
  for i in range(3):
    for j in range(3):
        pltText = '{:.2F}'.format(avgMatrix[i,j])
        plt.text(j,i, pltText, horizontalalignment='center', \
                   verticalalignment='center', size = 24)
  xlabels = ['Steels', 'Al Alloys', 'Plastics']
  ax.set_xticks(range(3))
  ax.set_xticklabels(xlabels, rotation = 90, size = 18)
  ax.set_yticks(range(3))
  ax.set_yticklabels(xlabels, size = 18)
  plt.savefig('./figures2D/averageDistanceMatrix.pdf',bbox_inches='tight')
drawDistanceMatrix()


## Plots the latent space in 2D with a Convex Hull

In [ ]:
# The latent field
def plotLatent(saveFileName):
    clrs = ['purple', 'green', 'red', 'blue', 'black', 'violet', 'cyan',]
    colorcol = dataIdentifier['classID']
    ptLabel = dataIdentifier['name']
    autoencoder = materialEncoder.vaeNet
    z = autoencoder.encoder.z.to('cpu').detach().numpy()
    fig, ax = plt.subplots()
    #List of meaterials to plot name
    matidxs = np.array([13,14,15,48,18,10,9,8,28,20,30,69,27,37,\
                        5,6,73,77,78,85,91,88,75,80,82,83]).astype(int)-2
    for i in range(np.max(colorcol)+1): 
        zMat = np.vstack((z[colorcol == i,0], z[colorcol == i,1])).T
        ax.scatter(zMat[:, 0], zMat[:, 1], marker='*', c = 'black', s = 8)#clrs[i]
        cent = np.mean(zMat, 0)   
        
        if(np.shape(zMat)[0]>3):
            hull = ConvexHull(zMat)
            pts = []
            for pt in zMat[hull.simplices]:
                pts.append(pt[0].tolist())
                pts.append(pt[1].tolist())

            pts.sort(key=lambda p: np.arctan2(p[1] - cent[1],
                                            p[0] - cent[0]))
            pts = pts[0::2]  # Deleting duplicates
            pts.insert(len(pts), pts[0])
            poly = Polygon(1.1*(np.array(pts)- cent) + cent,
                           facecolor= clrs[i], alpha=0.2, edgecolor = 'black') #'black'
            poly.set_capstyle('round')
            plt.gca().add_patch(poly)
        ax.annotate(dataIdentifier['className'][i], (cent[0], cent[1]), size = 15, c = 'red')
    for i, txt in enumerate(ptLabel):
      if(i in matidxs):
        ax.annotate(txt, (z[i,0], z[i,1]), size = 12)

    plt.xlabel('$z_0$'.format(0), size = 18)
    plt.ylabel('$z_1$'.format(1), size = 18)
    minor_ticks = np.arange(-2.5, 2.5, 0.1)
    ax.set_xticks(minor_ticks, minor=True)
    ax.set_yticks(minor_ticks, minor=True)

    plt.grid(which='both')
    plt.savefig(saveFileName)

    return fig, ax
  
plotLatent(saveFileName = './figures2D/latent.pdf');

In [ ]:
def plotLatentWithProperty(ltnt1 = 0, ltnt2 = 1):
  n = 80
  zmin, zmax = -2.5,2.5
  X,Y = np.meshgrid(np.linspace(zmin, zmax, n), np.linspace(zmin, zmax, n))
  Z = torch.zeros((n**2, vaeSettings['encoder']['latentDim']))
  Z[:,ltnt1], Z[:,ltnt2] = to_torch(X.reshape(-1)), to_torch(Y.reshape(-1))

  vae = materialEncoder.vaeNet
  trainData_z_np = to_np(vae.encoder.z)
  decoded = vae.decoder(Z)

  #-------------------------------------------#
  #props = ['youngsModulus','costPerKg','massDensity','yieldStrength']
  for p in props:
    idx = materialEncoder.dataInfo[p]['idx']
    scaleMax = materialEncoder.dataInfo[p]['scaleMax']
    scaleMin = materialEncoder.dataInfo[p]['scaleMin']

    matPropVal = 10.**(scaleMin + decoded[:,idx]*(scaleMax - scaleMin))

    fig, ax = plotLatent(saveFileName = './figures2D/swrksLatent.pdf')
    surf = ax.contourf(X, Y, np.log10(to_np(matPropVal).reshape((n,n))), levels = 20, cmap='coolwarm', alpha = 0.7)
    plt.clabel(surf, inline=False, fontsize=12, fmt ='%0.2F', colors = 'red')
    ax.set_xlabel('$z_0$')
    ax.set_ylabel('$z_1$')
    ax.set_title(p)
    cbar = plt.colorbar(surf)
    cbar.set_label('log10({:s})'.format(str(p)))
    plt.savefig('./figures2D/{:s}_latentField.pdf'.format(p), dpi=200, bbox_inches='tight')

  #-------------------------------------------#
  
plotLatentWithProperty()

In [ ]:
def plotTrueAndReconstructedDistribution():

  vae = materialEncoder.vaeNet
  trainData_z_np = to_np(vae.encoder.z)
  decodedVals = vae.decoder(vae.encoder.z)
  
  bw = 0.405
  fig, ax = plt.subplots(1,2)
  #-------------------------------------------#
  props = ['youngsModulus','costPerKg','massDensity','yieldStrength']
  props_red = ['youngsModulus','yieldStrength']
  for ctr, p in enumerate(props_red):
    idx = materialEncoder.dataInfo[p]['idx']
    scaleMax = materialEncoder.dataInfo[p]['scaleMax']
    scaleMin = materialEncoder.dataInfo[p]['scaleMin']

    matVal_decoded = 10.**(scaleMin + decodedVals[:,idx]*(scaleMax - scaleMin))
    matVal_data = 10.**(scaleMin + trainingData[:,idx]*(scaleMax - scaleMin))

    #sns.set_style('white')
    plt.subplot(1,2,ctr+1)
    f = sns.kdeplot(to_np(matVal_decoded), bw_adjust=bw, fill = True, alpha = 0.1, label='decoded')
    f = sns.kdeplot(to_np(matVal_data), bw_adjust=bw,  fill = True, alpha = 0.1, linestyle="--", label='actual')
    f.set(xlabel = p, ylabel = 'frequency',yticklabels=[])
    plt.legend()
    plt.axis('auto')
    plt.title(p)
  
  plt.savefig('./figures2D/trueAndReconstructedDistribution.pdf'.format(p), dpi=200, bbox_inches='tight')

plotTrueAndReconstructedDistribution()  

In [ ]:
def plotLatentPropertyWithGradients(ltnt1 = 0, ltnt2 = 1):
  n = 40
  zmin, zmax = -2.5,2.5
  X,Y = np.meshgrid(np.linspace(zmin, zmax, n), np.linspace(zmin, zmax, n))
  Z = torch.zeros((n**2, vaeSettings['encoder']['latentDim']))
  Z[:,ltnt1], Z[:,ltnt2] = to_torch(X.reshape(-1)), to_torch(Y.reshape(-1))
  Z = torch.tensor(Z, requires_grad = True)
  vae = materialEncoder.vaeNet
  decodedVals = vae.decoder(Z)



  fig, ax = plt.subplots(1,1)
  #-------------------------------------------#
  props = ['youngsModulus','costPerKg','massDensity','yieldStrength']
  props_red = ['yieldStrength']
  for ctr, p in enumerate(props_red):
    idx = materialEncoder.dataInfo[p]['idx']
    scaleMax = materialEncoder.dataInfo[p]['scaleMax']
    scaleMin = materialEncoder.dataInfo[p]['scaleMin']

    matVal_decoded = 10.**(scaleMin + decodedVals[:,idx]*(scaleMax - scaleMin))

    dE_dz = to_np(torch.autograd.grad(matVal_decoded, Z, grad_outputs = torch.ones(Z.shape[0]), create_graph = True)[0])
    U = dE_dz[:,0] / (1e-4+np.sqrt(dE_dz[:,0]**2 + dE_dz[:,1]**2))
    V = dE_dz[:,1] / (1e-4+np.sqrt(dE_dz[:,0]**2 + dE_dz[:,1]**2))
    plt.subplot(1,1,ctr+1)
    surf = plt.contourf(X, Y, np.log10(to_np(matVal_decoded).reshape((n,n))), levels = 100, cmap='coolwarm', alpha = 0.7)
    plt.quiver(X,Y,U,V, headwidth = 0, headlength = 0,headaxislength = 0, color = 'black')
#     plt.clabel(surf, inline=False, fontsize=12, fmt ='%0.2F', colors = 'black')
    plt.title(p)
    plt.xlabel('$z_0$')
    plt.ylabel('$z_1$')
    cbar = plt.colorbar(surf)
#     cbar.set_label('$log_{10}$({:s})'.format(p))
  
  plt.savefig('./figures2D/latentSpaceGradient.pdf'.format(p), dpi=200, bbox_inches='tight')
  
plotLatentPropertyWithGradients()